### Tutorial Notebook: G<sub>0</sub>W<sub>0</sub> Approximation 

In this notebook, we present an example calculation of quasiparticle energies using QuatumMASALA's `gw` module.

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# Imports
import numpy as np
import sys

sys.path.append(".")

dirname = "./"

### DFT Calculation

We will start with a DFT calculation to get the energy eigenfunctions and eigenvalues.

In [31]:
import numpy as np

from qtm.constants import RYDBERG, ELECTRONVOLT
from qtm.kpts import KList
from qtm.lattice import RealLattice
from qtm.crystal import BasisAtoms, Crystal
from qtm.pseudo import UPFv2Data
from qtm.gspace import GSpace
from qtm.mpi import QTMComm
from qtm.dft import DFTCommMod, scf

from qtm.io_utils.dft_printers import print_scf_status

from qtm import qtmconfig
from qtm.logger import qtmlogger
# qtmconfig.fft_backend = 'mkl_fft'

from mpi4py.MPI import COMM_WORLD
comm_world = QTMComm(COMM_WORLD)
dftcomm = DFTCommMod(comm_world)

# Lattice
reallat = RealLattice.from_alat(alat=10.2,  # Bohr
                                a1=[-0.5,  0.0,  0.5],
                                a2=[ 0.0,  0.5,  0.5],
                                a3=[-0.5,  0.5,  0.0])

# Atom Basis
si_oncv = UPFv2Data.from_file('Si_ONCV_PBE-1.2.upf')
si_atoms = BasisAtoms.from_alat('Si', si_oncv, 28.086, reallat,
                               [[0.875, 0.875, 0.875], [0.125, 0.125, 0.125]])

crystal = Crystal(reallat, [si_atoms, ])  # Represents the crystal


# Generating k-points from a Monkhorst Pack grid (reduced to the crystal's IBZ)
mpgrid_shape = (4, 4, 4)
x = np.linspace(0,1,mpgrid_shape[0], endpoint=False)
y = np.linspace(0,1,mpgrid_shape[1], endpoint=False)
z = np.linspace(0,1,mpgrid_shape[2], endpoint=False)
xx,yy,zz = np.meshgrid(x,y,z, indexing="ij")
kcryst = np.vstack([xx.flatten(),yy.flatten(),zz.flatten()])

kpts = KList(recilat=crystal.recilat, k_coords=kcryst, k_weights=np.ones(kcryst.shape[1])/kcryst.shape[1])


# -----Setting up G-Space of calculation-----
ecut_wfn = 25 * RYDBERG
ecut_rho = 4 * ecut_wfn
grho = GSpace(crystal.recilat, ecut_rho)
gwfn = grho

# -----Spin-polarized (collinear) calculation-----
is_spin, is_noncolin = False, False
mag_start = [0.0]
numbnd_occ = 4
numbnd_nscf = 30

occ = 'fixed'

conv_thr = 1E-8 * RYDBERG
diago_thr_init = 1E-2 * RYDBERG

### DFT: SCF calculation for occupied bands

In [32]:
from qtm.constants import ELECTRONVOLT_HART
from qtm.kpts import KList


kpts = KList(recilat=crystal.recilat, k_coords=kpts.k_cryst, k_weights=np.ones(kpts.k_cryst.shape[1])/kpts.k_cryst.shape[1])

scf_out = scf(dftcomm, crystal, kpts, grho, gwfn,
          numbnd_occ, is_spin, is_noncolin,
          rho_start=mag_start, occ_typ=occ,
          conv_thr=conv_thr, diago_thr_init=diago_thr_init,
          iter_printer=print_scf_status,
          ret_vxc=True)


print("SCF Routine has exited")
# print(qtmlogger)

Iteration # 1, Run Time:   3.3 sec
Convergence Status   : NOT Converged
SCF Error           : 5.5925e-02 Ry
Avg Diago Iterations: 8.0
Diago Threshold     : 1.00e-02 Ry

Total Energy:          -15.94105986 Ry

      one-el:            4.86451170 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.96856844 eV
----------------------------------------

Iteration # 1, Run Time:   4.4 sec
Convergence Status   : NOT Converged
SCF Error           : 6.0082e-02 Ry
Avg Diago Iterations: 2.8
Diago Threshold     : 6.99e-04 Ry

Total Energy:          -15.96485154 Ry

      one-el:            4.84072003 Ry
     Hartree:            0.80642513 Ry
          XC:           -4.71223800 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            5.91288694 eV
----------------------------------------

Iteration # 2, Run Time:   5.3 sec
Convergence Status   : NOT Converged
SCF Error           : 2.6353e-03 R

/home/agrim/codes/QuantumMASALA_presentation_25Sept2023/QuantumMASALA/src/qtm/dft/scf.py:420: ComplexWarning: Casting complex values to real discards the imaginary part
  hpsi_r = sum((v_xc * psi_r.get_density())).integrate_unitcell()


#### DFT: NSCF calculation for unshifted grid
Observe that `maxiter` has been set to `1` and `diago_thr_init` has been set to a high value.

In [33]:
rho = scf_out[1].copy()
nscf_out = scf(dftcomm, crystal, kpts, grho, gwfn,
          numbnd_nscf, is_spin, is_noncolin,
          rho_start=rho, 
          occ_typ=occ,
          conv_thr=conv_thr, 
          diago_thr_init=(conv_thr/crystal.numel)/10,
          iter_printer=print_scf_status,
          maxiter=1,
          ret_vxc=True)

scf_converged_nscf, rho_nscf, l_wfn_kgrp, en_nscf, vxc = nscf_out

Iteration # 1, Run Time:  21.1 sec
Convergence Status   :  Converged
SCF Error           : 2.2628e-10 Ry
Avg Diago Iterations: 20.0
Diago Threshold     : 1.25e-10 Ry

Total Energy:          -15.74868364 Ry

      one-el:            4.88766950 Ry
     Hartree:            1.10873528 Ry
          XC:           -4.84532973 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            6.47367697 eV
    LU Level:            7.11964564 eV
----------------------------------------



#### DFT: NSCF calculation for shifted grid

Dielectric matrix calculation for the $q\to 0$ point will require energy eigenfunctions for a slightly shifted $k$-grid.

In [35]:
k_coords_q = kpts.k_cryst+np.array([[0,0,0.001]]).T
k_weights_q = np.ones(k_coords_q.shape[1])/k_coords_q.shape[1]
kpts_q = KList(recilat=crystal.recilat, k_coords=k_coords_q, k_weights=k_weights_q)

rho = scf_out[1].copy()
out_q = scf(dftcomm, crystal, kpts_q, grho, gwfn,
          numbnd_nscf, is_spin, is_noncolin,
          rho_start=rho, 
          occ_typ=occ,
          conv_thr=conv_thr, 
          diago_thr_init=(conv_thr/crystal.numel)/10,
          iter_printer=print_scf_status,
          maxiter=1)

scf_converged_nscf_q, rho_nscf_q, l_wfn_kgrp_q, en_nscf_q = out_q


print("Shifted SCF Routine has exited")
# print(qtmlogger)

Iteration # 1, Run Time:  21.9 sec
Convergence Status   :  Converged
SCF Error           : 3.1062e-10 Ry
Avg Diago Iterations: 20.0
Diago Threshold     : 1.25e-10 Ry

Total Energy:          -15.74868281 Ry

      one-el:            4.88767032 Ry
     Hartree:            1.10873528 Ry
          XC:           -4.84532973 Ry
       Ewald:          -16.89975869 Ry

    HO Level:            6.47365312 eV
    LU Level:            7.11792828 eV
----------------------------------------

Shifted SCF Routine has exited


### Load Input Files
Input data is handled by the ``EpsInp`` class.\
The data can be provided either by constructing the ``EpsInp`` object or by reading BGW-compatible input file ``epsilon.inp``.\
The attributes have been supplied with docstrings from BerkeleyGW's input specification, so they will be accessible directly in most IDEs.

In [7]:
from qtm.gw.io_bgw.epsinp import Epsinp

# Constructing input manually
# epsinp = Epsinp(epsilon_cutoff=1.2,
#                 use_wfn_hdf5=True,
#                 number_bands=8,
#                 write_vcoul=True,
#                 qpts=[[0.0,0.0,0.0]],
#                 is_q0=[True])

# Reading from epsilon.inp file
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')
# print(epsinp)

# There is an analogous system to read SigmaInp
from qtm.gw.io_bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')
# print(sigmainp)

### Initialize Epsilon Class

``Epsilon`` class can be initialized by either directly passing the required `quantummasala.core` objects or by passing the input objects discussed earlier.

In [8]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon
from qtm.klist import KList

kpts_gw =   KList(recilat=kpts.recilat,   cryst=kpts.k_cryst.T,   weights=kpts.k_weights)
kpts_gw_q = KList(recilat=kpts_q.recilat, cryst=kpts_q.k_cryst.T, weights=kpts_q.k_weights)

# Manual initialization
epsilon = Epsilon(
    crystal = crystal,
    gspace = grho,
    kpts = kpts_gw,
    kptsq = kpts_gw_q,        
    l_wfn = l_wfn_kgrp,
    l_wfnq = l_wfn_kgrp_q,
    qpts = QPoints.from_cryst(kpts.recilat, epsinp.is_q0, *epsinp.qpts),
    epsinp = epsinp,
)

# epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 11800.40it/s]


The three main steps involved in the calculation have been mapped to the corresponding functions:
1.  ``matrix_elements``: Calculation of Planewave Matrix elements
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix

<!-- 1.  ``matrix_elements``: Calculation of Planewave Matrix elements
    $$M_{nn'}({\textbf k},{\textbf q},{\textbf G}) = \bra{n\,{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n'\,\textbf k}$$
    where the $\textbf G$-vectors included in the calculation satisfy $|\textbf q + \textbf G|^2 < E_{\text{cut}}$.
    Since this is a convolution in k-space, the time complexity can be reduced from $\mathcal{O}\left(N^2_G\right)$ to $\mathcal{O}\left(N_G\ln N_G\right)$ by using Fast Fourier Transform, where $N_G$  the number of reciprocal lattice vectors in the wavefunction.
    $$
    M_{nn'}({\bf k},{\bf q},\{{\bf G}\}) = {\rm FFT}^{-1}\left( \phi^{*}_{n,{\bf k}+{\bf q} }({\bf r}) \phi_{n',{\bf k} }({\bf r}) \right).
    $$
    where $\phi_{n',{\bf k}}({\bf r}) = {\rm FFT}\left( \psi_{n\bf k}(\bf G)\right)$. 
    
2.  ``polarizability``: Calculation of RPA polarizability matrix $P$
    $$
        P_{\textbf{GG'}}{\left({\textbf q}\;\!;0\right)}=
        \,\,{}\sum_{n}^{\textrm occ}\sum_{n'}^{\textrm emp}\sum_{{\textbf k}}
        \frac{
        \bra{n'\textbf k}e^{-i({\textbf q}+{\textbf G})\cdot{\textbf r}}\ket{n{\textbf k}{+}{\textbf q}}
        \bra{n{\textbf k}{+}{\textbf q}}e^{i({\textbf q}+{\textbf G'})\cdot{\textbf r}}\ket{n'\textbf k}
        }{E_{n{\textbf k}{+}{\textbf q}}\,{-}\,E_{n'{\textbf k}}}.
    $$
3.  ``epsilon_inverse``: Calculation of (static) epsilon-inverse matrix
    $$
        \epsilon_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}=
        \delta_{\textbf{GG'}}\,{-}\,v{\left({\textbf q}{+}{\textbf G}\right)} \,
        P_{\textbf{GG'}}{\left({\textbf q}\;\!\right)}
    $$
    where $ v(\textbf{q} + \textbf{G}) = \frac{8\pi}{\left|\textbf q + \textbf G\right|^2} $ is bare Coulomb potential, written in Rydberg units. If this formula is used as-is for the case where $|\textbf q| = |\textbf G| = 0$, the resulting $v\left({\textbf{q=0}, \textbf{G=0}}\;\!\right)$ blows up as $1/q^2$. However, for 3D gapped systems, the matrix elements $\big| M_{nn'}\left({\bf k},{\textbf{q}\to\textbf{0}},{\textbf{G=0}}\right)\big| \sim q$ cancel the Coulomb divergence and $\epsilon_{\textbf{00}}\left({\textbf q\to\textbf{0}}\;\!\right) \sim q^2/q^2$ which is a finite number. In order to calculate $\epsilon_{\textbf{00}}\left({\textbf q=\textbf{0}}\;\!\right)$, we use the scheme specified in BGW2012, wherein $q=0$ is replaced with a small but non-zero value. Since matrix element calculation involves the eigenvectors $\ket{n{\textbf k}{+}{\textbf q}}$, having a non-$\Gamma$-centered $\textbf q\to 0$ point requires mean-field eigenvectors over a shifted $k$-grid. -->

In [9]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)
        
        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv

        if i_q == epsilon.qpts.index_q0:
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsmats.append(epsinv)
            
        if False:        
            # Compare the results with BGW's results
            if i_q == epsilon.qpts.index_q0:
                epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            else:
                epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])    

            # Calculate stddev between reference and calculated epsinv matrices
            std_eps = np.std(np.abs(epsref) - np.abs(epsinv)) / np.sqrt(np.prod(list(epsinv.shape)))
        
            epstol = 1e-16
            if np.abs(std_eps) > epstol:
                print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")
                print(np.max(np.abs(epsinv-epsref)))
                indices = np.where(np.abs(epsinv)-np.abs(epsref)>1e-5)
                

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)

calculate_epsilon()


Epsilon> q-pt index: 100%|██████████| 64/64 [00:35<00:00,  1.79it/s]


### Sigma Calculation

Here we demonstate the calculation of diagonal matrix elements of $\Sigma_{\text{QP}}$. The input parameters for sigma calculation are being read from `sigma.inp` file, but the same parameters can also be provided by manually constructing a `SigmaInp` object. 

Here we will calculate $\bra{nk}\Sigma_{\text{QP}}\ket{nk}$ for the following k-points:
- $\Gamma$: `k=(0,0,0)`
- $L$: `k=(0.5,0.5,0)`
- $X$: `k=(0,0.5,0)`

In [10]:
from qtm.gw.sigma import Sigma

outdir = dirname+"temp/"


sigma = Sigma.from_qtm_scf(
    crystal = crystal,
    gspace = grho,
    kpts = kpts_gw,
    kptsq=kpts_gw_q,
    l_wfn_kgrp=l_wfn_kgrp,
    l_wfn_kgrp_q=l_wfn_kgrp_q,
    sigmainp=sigmainp,
    epsinp = epsinp,
    epsilon=epsilon,
    rho=rho,
    vxc=vxc
)

# Alternatively, the Sigma object can also be intitialized from pw2bgw.x output data (after being procesed by wfn2hdf5.x).
# sigma = Sigma.from_data(
#     wfndata=wfndata,
#     wfnqdata=wfnqdata,
#     sigmainp=sigmainp,
#     epsinp=epsinp,
#     l_epsmats=epsilon.l_epsinv,
#     rho=rho,
#     vxc=vxc,
#     outdir=outdir,
# )

Vcoul calculation for qpts: 100%|██████████| 64/64 [00:00<00:00, 13803.44it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x7f5d011bf190>
        * qpts = <qtm.gw.core.QPoints object at 0x7f5cf622b6d0>
        * bare_coulomb_cutoff = 10.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 64
        * vcoul = <class 'list'> of length 64
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts: 100%|██████████| 64/64 [01:02<00:00,  1.03it/s]


In [20]:
sigma_sx_cohsex_mat = sigma.sigma_sx_static(yielding=True)    
print("Sigma SX COHSEX")
sigma.pprint_sigma_mat(sigma_sx_cohsex_mat)

Sigma_SX_Static:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_Static: 100%|██████████| 64/64 [00:02<00:00, 25.60it/s]

Sigma SX COHSEX
  n      ik=1        ik=41       ik=9    
  1    11.875823   10.671036   11.328749
  2     8.032456   10.679500   10.014670
  3     8.030223    8.719398    8.365710
  4     8.031513    8.719927    8.366196
  5     2.906871    2.587371    2.872627
  6     2.912285    2.587355    2.426072
  7     2.911883    1.328943    2.427773
  8     2.681679    1.328864    0.739613


In [12]:
sigma_ch_cohsex_mat = sigma.sigma_ch_static()    
print("Sigma CH COHSEX")
sigma.pprint_sigma_mat(sigma_ch_cohsex_mat)

Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:05<00:00, 12.69it/s]

Sigma CH COHSEX
  n      ik=1        ik=41       ik=9    
  1    -6.627245   -6.302982   -6.494446
  2    -5.099631   -6.303241   -6.068626
  3    -5.099172   -5.571211   -5.407032
  4    -5.099275   -5.571192   -5.407508
  5    -4.744831   -4.701345   -4.917363
  6    -4.746989   -4.701008   -4.286926
  7    -4.746800   -3.460411   -4.287495
  8    -4.757448   -3.460265   -2.975076


In [24]:
sigma.autosave=False
sigma.print_condition=True
cohsex_result = sigma.calculate_static_cohsex()

Sigma_X: 100%|██████████| 64/64 [00:02<00:00, 25.12it/s]


Sigma X
  n      ik=0        ik=40       ik=8    
  1   -17.549893  -16.038862  -16.918923
  2   -12.974998  -16.038766  -14.908727
  3   -12.974855  -13.417514  -13.204429
  4   -12.974868  -13.417599  -13.204461
  5    -5.765277   -5.196146   -5.934487
  6    -5.765520   -5.196565   -5.081701
  7    -5.765733   -3.789600   -5.081712
  8    -5.831193   -3.789630   -2.406361


Sigma_SX_Static: 100%|██████████| 64/64 [00:02<00:00, 27.58it/s]


Sigma SX STATIC
  n      ik=0        ik=40       ik=8    
  1    11.875823   10.671036   11.328749
  2     8.032456   10.679500   10.014670
  3     8.030223    8.719398    8.365710
  4     8.031513    8.719927    8.366196
  5     2.906871    2.587371    2.872627
  6     2.912285    2.587355    2.426072
  7     2.911883    1.328943    2.427773
  8     2.681679    1.328864    0.739613


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:04<00:00, 15.39it/s]


Sigma CH STATIC
  n      ik=0        ik=40       ik=8    
  1    -6.627245   -6.302982   -6.494446
  2    -5.099631   -6.303241   -6.068626
  3    -5.099172   -5.571211   -5.407032
  4    -5.099275   -5.571192   -5.407508
  5    -4.744831   -4.701345   -4.917363
  6    -4.746989   -4.701008   -4.286926
  7    -4.746800   -3.460411   -4.287495
  8    -4.757448   -3.460265   -2.975076


Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:19<00:00,  3.22it/s]

Sigma CH EXACT STATIC
  n      ik=0        ik=40       ik=8    
  1    -7.444549   -7.623265   -7.626651
  2    -7.757747   -7.623061   -7.314287
  3    -7.756375   -7.476842   -7.673885
  4    -7.756953   -7.477383   -7.674314
  5    -7.333986   -6.794872   -7.354459
  6    -7.335867   -6.795333   -7.120309
  7    -7.335678   -7.674095   -7.120625
  8    -7.829754   -7.673887   -6.232160
Sig (Exact):
  n      ik=0        ik=40       ik=8    
  1   -13.118620  -12.991090  -13.216825
  2   -12.700290  -12.982327  -12.208345
  3   -12.701007  -12.174958  -12.512605
  4   -12.700308  -12.175056  -12.512579
  5   -10.192392   -9.403647  -10.416319
  6   -10.189102   -9.404542   -9.775938
  7   -10.189528  -10.134752   -9.774565
  8   -10.979268  -10.134653   -7.898908
Eqp0 (Exact):
  n      ik=0        ik=40       ik=8    
  1    -8.251757   -3.521091   -5.575113
  2     5.142806   -3.512327   -2.586290
  3     5.142088    2.015015    3.845947
  4     5.142788    2.014918    3.845973
  5  

In [25]:
sigma.print_condition=True
print("Sigma CH COHSEX EXACT")
sigma_ch_exact_mat = sigma.sigma_ch_static_exact()    
sigma.pprint_sigma_mat(sigma_ch_exact_mat)

Sigma CH COHSEX EXACT


Sigma_CH_Static_Exact:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_Static_Exact: 100%|██████████| 64/64 [00:20<00:00,  3.16it/s]

  n      ik=0        ik=40       ik=8    
  1    -7.444549   -7.623265   -7.626651
  2    -7.757747   -7.623061   -7.314287
  3    -7.756375   -7.476842   -7.673885
  4    -7.756953   -7.477383   -7.674314
  5    -7.333986   -6.794872   -7.354459
  6    -7.335867   -6.795333   -7.120309
  7    -7.335678   -7.674095   -7.120625
  8    -7.829754   -7.673887   -6.232160


In [26]:
sigma.print_condition=False
sigma_sx_gpp = sigma.sigma_sx_gpp()    
print("Sigma SX GPP")
sigma.pprint_sigma_mat(sigma_sx_gpp)

Sigma_SX_GPP:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_SX_GPP: 100%|██████████| 64/64 [00:50<00:00,  1.28it/s]

Sigma SX GPP
  n      ik=0        ik=40       ik=8    
  1    12.243984   10.838378   11.549620
  2     8.179767   10.845757   10.174672
  3     8.176508    8.837621    8.500791
  4     8.177501    8.838613    8.501592
  5     3.289017    2.865594    3.299221
  6     3.299458    2.866378    2.790230
  7     3.300086    1.738249    2.792023
  8     3.367547    1.737804    1.089953


In [27]:
sigma.print_condition=False
sigma_ch_gpp,_ = sigma.sigma_ch_gpp()    
print("Sigma CH GPP")
sigma.pprint_sigma_mat(sigma_ch_gpp)

Sigma_CH_GPP:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_CH_GPP: 100%|██████████| 64/64 [00:54<00:00,  1.17it/s]

Sigma CH GPP
  n      ik=0        ik=40       ik=8    
  1    -5.796967   -5.846745   -5.879214
  2    -5.404032   -5.849259   -5.642407
  3    -5.402759   -5.559027   -5.572849
  4    -5.402481   -5.559464   -5.573498
  5    -5.334594   -4.982419   -5.446964
  6    -5.342859   -4.983354   -4.890361
  7    -5.343519   -4.622291   -4.891108
  8    -5.720043   -4.621877   -3.772002


In [28]:
gpp_result = sigma.calculate_gpp()

Sigma_X:   0%|          | 0/64 [00:00<?, ?it/s]

Sigma_X: 100%|██████████| 64/64 [00:02<00:00, 24.92it/s]

Sigma X GPP


  n      ik=0        ik=40       ik=8    
  1   -17.549893  -16.038862  -16.918923
  2   -12.974998  -16.038766  -14.908727
  3   -12.974855  -13.417514  -13.204429
  4   -12.974868  -13.417599  -13.204461
  5    -5.765277   -5.196146   -5.934487
  6    -5.765520   -5.196565   -5.081701
  7    -5.765733   -3.789600   -5.081712
  8    -5.831193   -3.789630   -2.406361


Sigma_CH_Static_Partial: 100%|██████████| 64/64 [00:05<00:00, 11.81it/s]

...

Sigma_CH_GPP: 100%|██████████| 64/64 [00:57<00:00,  1.12it/s]

Sigma CH GPP
  n      ik=0        ik=40       ik=8    
  1    -5.796967   -5.846745   -5.879214
  2    -5.404032   -5.849259   -5.642407
  3    -5.402759   -5.559027   -5.572849
  4    -5.402481   -5.559464   -5.573498
  5    -5.334594   -4.982419   -5.446964
  6    -5.342859   -4.983354   -4.890361
  7    -5.343519   -4.622291   -4.891108
  8    -5.720043   -4.621877   -3.772002
Sig GPP:
  n      ik=0        ik=40       ik=8    
  1   -11.511529  -11.707371  -11.814619
  2   -11.528322  -11.702178  -10.999293
  3   -11.529708  -11.091736  -11.409914
  4   -11.528687  -11.091546  -11.409770
  5    -9.105431   -8.359735   -9.300777
  6    -9.103361   -8.360703   -8.598523
  7    -9.103605   -8.780485   -8.597363
  8    -9.719842   -8.780515   -6.716952
Eqp0


  n      ik=0        ik=40       ik=8    
  1    -6.644666   -2.237372   -4.172907
  2     6.314774   -2.232179   -1.377238
  3     6.313388    3.098238    4.948638
  4     6.314409    3.098428    4.948782
  5     9.982196    7.767543    8.876899
  6     9.984266    7.766576   10.870412
  7     9.984022   18.518468   10.871572
  8    11.226026   18.518438   15.216467


Sigma_SX_GPP: 100%|██████████| 64/64 [00:47<00:00,  1.35it/s]

Sigma CH GPP dE
  n      ik=0        ik=40       ik=8    
  1    -6.056249   -6.123600   -6.149619
  2    -5.638186   -6.126359   -5.916335
  3    -5.637478   -5.843310   -5.860758
  4    -5.641839   -5.843758   -5.861447
  5    -5.560907   -5.286730   -5.772165
  6    -5.570104   -5.287019   -5.099012
  7    -5.570705   -4.909229   -5.100760
  8    -5.941092   -4.908886   -4.063214
Sigma SX GPP dE
  n      ik=0        ik=40       ik=8    
  1    -6.056249   -6.123600   -6.149619
  2    -5.638186   -6.126359   -5.916335
  3    -5.637478   -5.843310   -5.860758
  4    -5.641839   -5.843758   -5.861447
  5    -5.560907   -5.286730   -5.772165
  6    -5.570104   -5.287019   -5.099012
  7    -5.570705   -4.909229   -5.100760
  8    -5.941092   -4.908886   -4.063214
Z:
  n      ik=0        ik=40       ik=8    
  1     0.714882    0.775109    0.755522
  2     0.839959    0.775179    0.778936
  3     0.839968    0.817130    0.829067
  4     0.839972    0.817152    0.829063
  5     0.830712   